In [3]:
!pip install PyMuPDF

  Obtaining dependency information for PyMuPDF from https://files.pythonhosted.org/packages/4a/26/8c72973b8833a72785cedc3981eb59b8ac7075942718bbb7b69b352cdde4/pymupdf-1.26.3-cp39-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.7 MB 960.0 kB/s eta 0:00:20
   ---------------------------------------- 0.2/18.7 MB 1.5 MB/s eta 0:00:13
    --------------------------------------- 0.4/18.7 MB 2.7 MB/s eta 0:00:07
   - -------------------------------------- 0.9/18.7 MB 4.7 MB/s eta 0:00:04
   --- ------------------------------------ 1.7/18.7 MB 7.6 MB/s eta 0:00:03
   ------ --------------------------------- 3.2/18.7 MB 11.3 MB/s eta 0:00:02
   ---------- ----------------------------- 5.1/18.7 MB 15.5 MB/s eta 0:00:01
   --------------- ------------------------ 7.2/18.7 MB 19.1 MB/s eta 0:00:01
   ------------------- -------------------- 9.2/18.7 MB 22.7 MB/s eta 0:00:01
   -------------------- 

In [ ]:
!pip uninstall fitz